In [1]:
import robotic as ry
import numpy as np
import random
import time
from bridgebuild_env import create_n_boxes
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import robotic_execution as robex

### Task Env: Bridge Builder

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C = C, N = n_boxes, position = 'fixed')

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box1 = "box1";
box2 = "box2";
box3 = "box3";
table = "table";
boxSize = C.frame(box1).getSize()

C.view()

0

In [3]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
boxes = [box1, box2, box3]
last_box_memory = []
for l in range(3):
        box = np.random.choice(boxes)
        qStart = C.getJointState()
        graspDirection = 'x' #random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = 'z' #random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, accumulated_collisions=True)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        print('place direction:', placeDirection)

        if l > 1:

            offset_xyz = [0.0, 0.0, 0.00]
            poses_from_memory = np.vstack( (C.getFrame(last_box_memory[-2]).getPosition(), C.getFrame(last_box_memory[-1]).getPosition()) )
            middle_between_towers = np.mean(poses_from_memory, 0) + offset_xyz

            dir_vec = C.getFrame(last_box_memory[-2]).getPosition() - C.getFrame(last_box_memory[-1]).getPosition()
            pos2 = C.getFrame(last_box_memory[-2]).getPosition()
            pos1 = C.getFrame(last_box_memory[-1]).getPosition()
            rel_pos =  (pos1 + pos2) / 2 + np.array([0, 0, 0.08])
            
            dir_vec /= np.linalg.norm(dir_vec)

            M.komo.addObjective([2.], ry.FS.position, [box], ry.OT.eq, [1e3], rel_pos)
            M.komo.addObjective([2.], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
            M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])


            
        if l == 1:
            M.place_box(2., box, table, palm, placeDirection)

            M.komo.addObjective([2.], ry.FS.negDistance, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [-.04])
            M.komo.addObjective([2.], ry.FS.scalarProductXY, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [0])
            M.komo.addObjective([2.], ry.FS.positionRel, [box, last_box_memory[-1]], ry.OT.eq, [1.], [.0, .0, .1])
            M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)

        M.no_collision([], palm, table)
        # keep distance between boxes when placed defined by margin
        # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
        # place inital block as chosen by np.random.choice at a random location
        if l == 0:
            M.place_box(2., box, table, palm, placeDirection)

            M.target_relative_xy_position(2., box, table, [np.random.uniform(0.0, 0.3), np.random.uniform(0.0, 0.3)])
            M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)


        ways = M.solve()
        
        if not M.feasible:
            continue
        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box)
        
        M2.retract([.0, .2], gripper, dist = .13)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)

        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)
        # update memory
        last_box_memory.append(box)
        # remove box that was previously used
        boxes.remove(box)
        # update positions
        boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]


del M

=== placement 0: grasp x place z
place direction: z


AttributeError: 'ManipulationModelling' object has no attribute 'no_collision'

### On real robot: BotOp

In [1]:
import robotic as ry
import numpy as np
import random
import time
from bridgebuild_env import create_n_boxes
# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip
import robotic_execution as robex

In [2]:
def fetch_name_error_type(data):
    name_error_type_list = []
    for key, value in data.items():
        if isinstance(value, dict) and 'name' in value and 'err' in value and 'type' in value:
            name_error_type_list.append((value['name'], value['err'], value['type']))
    return name_error_type_list

In [3]:
def print_name_error_type(data):
    for name, error,  obj_type in data:
        print("Name:", name, "Type:", obj_type, "Error:", error)

In [4]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C = C, N = n_boxes, position = 'fixed')

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box1 = "box1";
box2 = "box2";
box3 = "box3";
table = "table";
boxSize = C.frame(box1).getSize()

C.view()

0

In [5]:
C.view()
bot = robex.Robot(C, on_real=False)
bot.goHome(C)

In [6]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
boxes = [box1, box2, box3]
last_box_memory = []
paths = []
vis = 1
n_success = 0
boxz = [box1, box3, box2]
while n_success <= 2:
    box = np.random.choice(boxes)
    # box = boxz[n_success]
    qStart = C.getJointState()
    graspDirection = random.choice(['x', 'y', 'z'])
    placeDirection = 'z' #random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
    info = f'placement {n_success}: grasp {graspDirection} place {placeDirection}'
    print('===', info)

    M = manip.ManipulationModelling(C, info, helpers=[gripper])
    M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e0, accumulated_collisions=True)
    M.grasp_box(0.5, gripper, box, palm, graspDirection)
    print('place direction:', placeDirection)

    if n_success > 1:
        print(f"3rd Block: ({box})")
        dir_vec = C.getFrame(last_box_memory[-2]).getPosition() - C.getFrame(last_box_memory[-1]).getPosition()
        pos2 = C.getFrame(last_box_memory[-2]).getPosition()
        pos1 = C.getFrame(last_box_memory[-1]).getPosition()
        rel_pos =  (pos1 + pos2) / 2 + np.array([0, 0, 0.08])
        dir_vec /= np.linalg.norm(dir_vec)

        #M.komo.addObjective([.3, .7], ry.FS.negDistance, [gripper, table], ry.OT.ineq, [1e1], [-0.15])
        M.komo.addObjective([2.], ry.FS.position, [box], ry.OT.eq, [1e1], rel_pos)
        #M.komo.addObjective([1.0], ry.FS.vectorY, [box], ry.OT.eq, [1e1], dir_vec)
        M.komo.addObjective([2.], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)
        M.komo.addObjective([2.], ry.FS.vectorZ, [gripper], ry.OT.eq, [1e1], [0, 0, 1])
        #M.komo.addObjective([0.3, 1.5], ry.FS.negDistance, [gripper, last_box_memory[-2]], ry.OT.ineq, [1e1], [-0.001])
        #M.komo.addObjective([0.3, 1.5], ry.FS.negDistance, [gripper, last_box_memory[-1]], ry.OT.ineq, [1e1], [-0.001])
        
    if n_success == 1:
        print(f"2nd Block: ({box})")
        M.place_box(2., box, table, palm, placeDirection)
        M.komo.addObjective([2.], ry.FS.negDistance, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [-.04])
        M.komo.addObjective([2.], ry.FS.scalarProductXY, [box, last_box_memory[-1]], ry.OT.eq, [1e1], [0])
        #M.komo.addObjective([2.], ry.FS.positionRel, [box, last_box_memory[-1]], ry.OT.eq, [1.], [.0, .0, .1])
        M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)

    M.keep_distance([], palm, table)
    # keep distance between boxes when placed defined by margin
    # x, y, z, smooth - table size: array([2.5 , 2.5 , 0.1 , 0.02])
    # place inital block as chosen by np.random.choice at a random location
    if n_success == 0:
        print(f"1st Block: ({box})")
        M.place_box(2., box, table, palm, placeDirection)
        initial_pos = [np.random.uniform(0.0, 0.1), np.random.uniform(0.0, 0.1)]
        # initial_pos= [0.020433060890705013, 0.08433819136314634]
        print("Trying xy pos: ", initial_pos)
        M.target_relative_xy_position(2., box, table, initial_pos)
        M.auto_pairwise_keep_dist([], keep_dist_between, margin = 0.08)

    ways = M.solve()

    if not M.feasible:
        print(f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M :(")
        continue
    print("Submotion 0")
    M2 = M.sub_motion(0)
    # M = manip.ManipulationModelling(C, info, helpers=[gripper])
    # M.setup_point_to_point_motion(qStart, ways[0])
    M2.keep_distance([.3,.7], palm, box)
    
    M2.retract([.0, .2], gripper, dist = .13)
    M2.approach([.8, 1.], gripper)
    paths.append(M2.solve())
    
    if not M2.feasible:
        print(f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M2 :(")
        continue

    M3 = M.sub_motion(1)
    #manip.ManipulationModelling(C, info)
    # M.setup_point_to_point_motion(ways[0], ways[1])
    M3.keep_distance([], table, box)
    if n_success > 1:
        M3.retract([.0, .2], gripper, dist = .13)
        M3.approach([.8, 1.], gripper)

    paths.append(M3.solve())
    if not M3.ret.feasible:
        print(f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M3 :(")
        continue
    
    M_report = M.komo.report(True)
    M_report = fetch_name_error_type(M_report)
    print_name_error_type(M_report)

    M2_report = M2.komo.report(True)
    M2_report = fetch_name_error_type(M2_report)
    print_name_error_type(M2_report)

    M3_report = M3.komo.report(True)
    M3_report = fetch_name_error_type(M3_report)
    print_name_error_type(M3_report)

    if vis == 0:
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

    bot.execute_path_blocking(C, path = paths[-2])
    C.attach(gripper, box)
    bot.grasp(C)

    bot.execute_path_blocking(C, path = paths[-1])
    C.attach(table, box)
    bot.release(C)
    # update memory
    last_box_memory.append(box)
    # remove box that was previously used
    boxes.remove(box)
    # update positions
    boxes_pos = [C.frame(box1).getPosition(), C.frame(box2).getPosition(), C.frame(box3).getPosition()]
    n_success += 1
        
del M

=== placement 0: grasp x place z
place direction: z
1st Block: (box2)
Trying xy pos:  [0.04130301085160684, 0.04408252892441109]
  -- feasible:placement 0: grasp x place z
     { time: 0.134746, evals: 200, done: 1, feasible: 1, sos: 5.09983, f: 0, ineq: 0.139115, eq: 0.359091 }
Submotion 0
  -- infeasible:sub_motion_0--placement 0: grasp x place z
     { time: 0.107807, evals: 40, done: 1, feasible: 0, sos: 2.95359, f: 0, ineq: 1.24732e-07, eq: 0.673886 }
Failed placing block number 1 with grasp direction x at M2 :(
=== placement 0: grasp z place z
place direction: z
1st Block: (box1)
Trying xy pos:  [0.03514775774073544, 0.03352417143788172]
  -- infeasible:placement 0: grasp z place z
     { time: 0.127945, evals: 201, done: 1, feasible: 0, sos: 2.95627, f: 0, ineq: 0.362882, eq: 1.22616 }
Failed placing block number 1 with grasp direction z at M :(
=== placement 0: grasp z place z
place direction: z
1st Block: (box1)
Trying xy pos:  [0.04964673330414428, 0.05548623548527157]
  -- i

KeyboardInterrupt: 